In [17]:
from google_auth_oauthlib.flow import InstalledAppFlow
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
creds = flow.run_local_server(port=0)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=485183065000-pj0cc7gq9heeo1n1c3o536h1nta5uebc.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A50459%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=EMkPJW0r7FlZeDtFV20yiPJITX2q0k&access_type=offline


In [19]:
from googleapiclient.discovery import build
service = build('gmail', 'v1', credentials=creds)

# list the latest N message IDs
results = service.users().messages().list(userId='me', maxResults=200).execute()
msgs = results.get('messages', [])

In [21]:
msgs

[{'id': '1964fde979c5f14d', 'threadId': '1964fde979c5f14d'},
 {'id': '1964f7a19618e040', 'threadId': '1964f7a19618e040'},
 {'id': '1964f641dfbc737e', 'threadId': '1964f641dfbc737e'},
 {'id': '1964f1f8f74081e1', 'threadId': '1964f1f8f74081e1'},
 {'id': '1964ee884b2e551a', 'threadId': '1964ee884b2e551a'},
 {'id': '1964e7a0320c4e63', 'threadId': '1964e7a0320c4e63'},
 {'id': '1964df8e7293aedd', 'threadId': '1964df8e7293aedd'},
 {'id': '1964d621fd20564f', 'threadId': '1964d621fd20564f'},
 {'id': '1964d3dc75869d9f', 'threadId': '1964d3dc75869d9f'},
 {'id': '1964d35ab34fcfb1', 'threadId': '1964d35ab34fcfb1'},
 {'id': '1964cd393ef3d308', 'threadId': '1964cd393ef3d308'},
 {'id': '1964ca205e639ef0', 'threadId': '1964ca205e639ef0'},
 {'id': '1964c8238d8a1ea3', 'threadId': '1964c8238d8a1ea3'},
 {'id': '1964c81ac76409ef', 'threadId': '1964c81ac76409ef'},
 {'id': '1964c800ef92ec58', 'threadId': '1964c800ef92ec58'},
 {'id': '1964c75b115d41fd', 'threadId': '1964c75b115d41fd'},
 {'id': '1964c7512bb8bdb

In [25]:
def get_raw_text(msg_id):
    msg = service.users().messages().get(userId='me', id=msg_id, format='full').execute()
    parts = msg['payload'].get('parts', [])
    # find the text/plain part, decode from base64…
    return extract_and_decode(parts)

In [27]:
import json

with open('train.jsonl','w') as out:
  for id in msgs:
    text = get_raw_text(id)
    label = label_lookup[id]   # however you store your ground‑truth
    example = {
      "messages":[
        {"role":"system","content":"You are an email classifier. Possible labels: …"},
        {"role":"user","content": text},
        {"role":"assistant","content": label}
      ]
    }
    out.write(json.dumps(example) + "\n")

HttpError: <HttpError 400 when requesting https://gmail.googleapis.com/gmail/v1/users/me/messages/%7B%27id%27%3A%20%271964fde979c5f14d%27%2C%20%27threadId%27%3A%20%271964fde979c5f14d%27%7D?format=full&alt=json returned "Invalid id value". Details: "[{'message': 'Invalid id value', 'domain': 'global', 'reason': 'invalidArgument'}]">